# Perform Retraining of DAGMNet

## Load Model

In [1]:
import scipy
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import tensorflow
import random

In [2]:
DAGMNet_ch2_name =  os.path.join("D:\ADS_Algorithm\ADSv1.3\data\Trained_Nets", 'DAGMNet_CH2.h5')

In [3]:
DAGMNet_ch2 = tensorflow.keras.models.load_model(DAGMNet_ch2_name, compile=False)

In [4]:
DAGMNet_ch2.input

<KerasTensor: shape=(None, 96, 112, 48, 2) dtype=float32 (created by layer 'input_1')>

In [5]:
swi_sample_img = nib.load("D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\SWI_Train\\2018-104_01-10113-D0MR_9_Ax_T2_GRE__SkullStripped_Training_Normalized.nii.gz")
swi_sample_data = swi_sample_img.get_fdata()
tof_sample_img = nib.load("D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\TOF3D_Train\\2018-104_01-10113-D0MR_6_3D_TOF_LARGE__SkullStripped_Training_Normalized.nii.gz")
tof_sample_data = tof_sample_img.get_fdata()

In [6]:
swi_sample_data = swi_sample_data[:,:,:,np.newaxis]
tof_sample_data = tof_sample_data[:,:,:,np.newaxis]

In [7]:
dagmnet_input = np.expand_dims(np.concatenate((swi_sample_data,tof_sample_data),axis=3), axis=0)

In [8]:
dagmnet_output = DAGMNet_ch2.predict(dagmnet_input)

1/1 [==============================] - 5s 5s/step


In [49]:
DAGMNet_ch2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 112, 48  0           []                               
                                , 2)]                                                             
                                                                                                  
 conv3d (Conv3D)                (None, 96, 112, 48,  1760        ['input_1[0][0]']                
                                 32)                                                              
                                                                                                  
 tf_op_layer_strided_slice (Ten  (None, 48, 56, 24,   0          ['input_1[0][0]']                
 sorFlowOpLayer)                2)                                                            

## Read-in Training Images

In [19]:
swi_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\SWI_Train"
tof_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\TOF3D_Train"

swi_files = [file for file in os.listdir(swi_dir) if file.endswith('.nii.gz')]
tof_files = [file for file in os.listdir(tof_dir) if file.endswith('.nii.gz')]

X_train = np.zeros((len(swi_files), 96, 112, 48, 2))
index = 0

for swi_file, tof_file in zip(swi_files,tof_files):
    swi_img = nib.load(os.path.join(swi_dir, swi_file))
    swi_data = swi_img.get_fdata()
    tof_img = nib.load(os.path.join(tof_dir, tof_file))
    tof_data = swi_img.get_fdata()

    swi_data = swi_data[:,:,:,np.newaxis]
    tof_data = tof_data[:,:,:,np.newaxis]
    input_array = np.concatenate((swi_data,tof_data),axis=3)
    X_train[index] = input_array
    index += 1

In [10]:
mask_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\MASK_Train"

mask_files = [file for file in os.listdir(mask_dir) if file.endswith('.nii.gz')]

y_train = np.zeros((len(mask_files), 96, 112, 48))
index = 0

for mask_file in mask_files:
    mask_img = nib.load(os.path.join(mask_dir, mask_file))
    mask_data = mask_img.get_fdata()
    y_train[index] = mask_data
    index += 1

## Separate Test Batch of Images

In [8]:
mask_train_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\MASK_Train"
mask_test_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\MASK_Test"
swi_train_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\SWI_Train"
swi_test_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\SWI_Test"
tof_train_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\TOF3D_Train"
tof_test_dir = "D:\\data_processed_ETIS\\Resized_Images\\DAGMNet_Training\\TOF3D_Test"

In [9]:
random.seed(777)

In [10]:
test_indexes = random.sample(range(len(os.listdir(mask_train_dir))), 100)

In [11]:
file_list = [os.listdir(mask_train_dir)[index] for index in test_indexes]
for file in file_list:
    os.rename(os.path.join(mask_train_dir, file), os.path.join(mask_test_dir, file))

In [12]:
file_list = [os.listdir(swi_train_dir)[index] for index in test_indexes]
for file in file_list:
    os.rename(os.path.join(swi_train_dir, file), os.path.join(swi_test_dir, file))

In [13]:
file_list = [os.listdir(tof_train_dir)[index] for index in test_indexes]
for file in file_list:
    os.rename(os.path.join(tof_train_dir, file), os.path.join(tof_test_dir, file))

## Define Custom Loss Functions

In [87]:
def l1_loss(y_pred):
    batch_size = tensorflow.shape(y_pred)[0]
    return tensorflow.reduce_sum(tensorflow.abs(y_pred)) / tensorflow.cast(batch_size, tensorflow.float64)

In [108]:
def dice_loss(y_true, y_pred):
    # tensorflow.keras.losses.Dice()
    batch_size = y_true.shape[0]
    # global glob_y_true
    # glob_y_true = y_true
    #print(1/batch_size)
    return (1 - (2 * tensorflow.reduce_sum(y_true * y_pred)) / (tensorflow.reduce_sum(y_true) + tensorflow.reduce_sum(y_pred))) / tensorflow.cast(batch_size, tensorflow.float64)

In [109]:
def dice_loss_mean(y_true, y_pred):
    # tensorflow.keras.losses.Dice()
    batch_size = y_true.shape[0]
    #print(1/batch_size)
    return (1 - (2 * tensorflow.reduce_mean(y_true * y_pred)) / (tensorflow.reduce_mean(y_true) + tensorflow.reduce_mean(y_pred)))

In [89]:
def combined_loss(y_true, y_pred):
    cross_entropy_loss = tensorflow.keras.losses.BinaryCrossentropy()
    weight_cross_entropy, weight_dice, weight_l1_regularization = (1.,1.,0.00001)
    
    return weight_cross_entropy * cross_entropy_loss(y_true, y_pred) + weight_dice * dice_loss(y_true, y_pred) + weight_l1_regularization * l1_loss(y_pred)

In [73]:
np.array([0.,1.]).dtype

dtype('float64')

In [57]:
y_train.shape

(404, 96, 112, 48)

In [74]:
test_pred = np.random.choice(np.array([0.,1.]),(404,96,112,48), p=[0.999998,0.000002])

In [110]:
dice_loss(y_train, test_pred)

<tf.Tensor: shape=(), dtype=float64, numpy=0.0024752475247524753>

In [111]:
dice_loss_mean(y_train, test_pred)

<tf.Tensor: shape=(), dtype=float64, numpy=1.0>

In [69]:
tensorflow.reduce_sum(y_train)

<tf.Tensor: shape=(), dtype=float64, numpy=352.0>

In [70]:
tensorflow.reduce_sum(test_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=408>

In [92]:
tensorflow.shape(y_train)[0]

<tf.Tensor: shape=(), dtype=int32, numpy=404>

In [107]:
tensorflow.print(glob_y_true)

AttributeError: 'Tensor' object has no attribute '_datatype_enum'

## Restructure Network

In [50]:
last_layer_to_keep = DAGMNet_ch2.layers[-5].output

In [51]:
New_DAGMNet_ch2 = tensorflow.keras.Model(inputs=DAGMNet_ch2.input, outputs=last_layer_to_keep)

In [52]:
New_DAGMNet_ch2.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.0003), loss=combined_loss, metrics=["accuracy"])

In [102]:
New_DAGMNet_ch2.fit(X_train, y_train, epochs=20, validation_split=0.2)

Epoch 1/20


ValueError: in user code:

    File "C:\Users\wijflo\tensorflow_gpu_env\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\wijflo\AppData\Local\Temp\ipykernel_11632\1514347977.py", line 5, in combined_loss  *
        return weight_cross_entropy * cross_entropy_loss(y_true, y_pred) + weight_dice * dice_loss(y_true, y_pred) + weight_l1_regularization * l1_loss(y_pred)
    File "C:\Users\wijflo\AppData\Local\Temp\ipykernel_11632\3381169296.py", line 7, in dice_loss  *
        return (1 - (2 * tensorflow.reduce_sum(y_true * y_pred)) / (tensorflow.reduce_sum(y_true) + tensorflow.reduce_sum(y_pred))) / tensorflow.cast(batch_size, tensorflow.float64)

    ValueError: None values not supported.


In [ ]:
New_DAGMNet_ch2.save("D:\\ADS_Inference\\Retrained_ADS\\Retrained_DAGMNet_ch2_404.h5")